# Reinforcement Learning with GridWorld

This is the first iPython notebook from my series on Reinforcement Learning (RL).
In this notebook I will try to explain some notions of Tabular Solution Methods for RL. In order to do so I will closely follow the second edition of an amazing book by Sutton and Barto on Reinforcement Learning; a draft of the book can be downloaded [Here](http://incompleteideas.net/book/the-book-2nd.html). The content of this notebook roughly corresponds to the main ideas presented in much more detail in the first part, Tabular Solution Methods, of the above book. Some of the presented algorithms are implemented in the examples, usually following the theoretical explanation. For the examples to run you need to download GridWorld.py and StateSpace.py files, and place them under the same directory as this notebook. The dependencies are NumPy, PyGame, and of course iPython.

## Basics

I assume that my environment is a finite **Marcov Decision Process** (MDP); in case you don't recall what MDP is, skimming through the third chapter of the above mentioned book should be more than enough to get you going. The assumption of MDP is a reasonable one since I will work only with the GridWorld example.

In the RL, there are six main notions I will need going forward, namely agent, environment, policy, reward, value function, and model. The first two don't need much of an explanation. The next one, **policy**, is formally a set of principles guiding one's actions to achieve desired outcomes. Or how a mathematician would put it, it is a map from the set of states of the environment to the agent's actions. Rewards lie at the heart of the RL, they define the goals of an agent. To be precise, a **reward** is a number returned from the environment to the agent after performing an action. The goal of an agent is to maximise the total reward it receives over the long run. The **value function**, a map from the set of states (and possibly actions of the states) to the real numbers, helps an agent to do precisely this; it indicates the long run desirability of being in a certain state. As opposed to the reward, which indicates the immediate desirability of being in that state. The last element of the above list, **model**, is an agent's internal model of the environment. Models are used for planning and will perhaps be considered much later when I come to planning (TODO).

There are two types of value functions:
* **state-value function**, $v_{\pi}(s)$, a map from the space of states to the real numbers, indicating the possible future (discounted) rewards obtained when following a particular policy $\pi$.
* **action-value function**, $q_{\pi}(s, a)$, a map from the space of states and their actions to the real numbers, indicating the possible future (discounted) rewards obtained when first selecting an action $a$ and thereafter following a particular policy $\pi$.

The next notion, needing our attention, is the **Bellman optimality equation** for:

* state-value function:
\begin{equation}
\begin{split}
v_{*}(s) =& \max_{a} \mathbb{E} \big[R_{t+1} + \gamma v_{*}(S_{t+1}) \,\big|\, S_{t} = s, A_{t} = a \big] \\
       =& \max_{a} \sum_{s^{\prime}, r} p(s^{\prime}, r \,|\, s, a) \big[r + \gamma v_{*}(s^{\prime}) \big] \,,
\end{split}
\end{equation}

* action-value function:
\begin{equation}
\begin{split}
q_{*}(s,a) =& \mathbb{E} \big[R_{t+1} + \gamma \max_{a^{\prime}} q_{*}(S_{t+1}, a^{\prime}) \,\big|\, S_{t} = s, A_{t} = a \big] \\
         =& \sum_{s^{\prime}, r} p(s^{\prime}, r \,|\, s, a) \big[r + \gamma \max_{a^{\prime}} q_{*}(s^{\prime}, a^{\prime}) \big] \,.
\end{split}
\end{equation}

Where $\mathbb{E}[\cdots]$ denotes the expected value of the expression in brackets, $p(s^{\prime}, r \,|\, s, a)$ indicates the transition probability to the state $s^{\prime}$ with reward $r$ when in state $s$ one chooses the action $a$; $\gamma$ is the discount factor. The reward received at the time step $t+1$ is represented by $R_{t+1}$; I use a capital character since this is a statistical variable as opposed to $r$ which denotes a concrete reward value. Both these equations are recursive; and as argued in the Sutton and Barto book, the Bellman optimality equation says that the value of a state under an optimal policy must be equal to the expected return for the best action from that state. Or in other words, if one knows the optimal value function for a state $s^{\prime}$, then the optimal value function for a preceeding state $s$ is exactly the above Bellman optimality equation.

So far so good; now, let me move to dynamic programming.

## Dynamic Programming

The general idea for solving RL problems, also known as **optimal control problems**, is to use a value function to organise the search for desirable policies. The first such a method usually presented is called **dynamic programming** (DP). DP denotes a set of methods for solving RL problems by solving the Bellman equation (defined below). Although DP methods are not very suitable for RL because they assume a perfect model of the environment as an MDP, they provide essential foundations for understanding more suitable methods.

The problem of finding the optimal policy in terms of DP methods splits into two separate sub-problems. The first, **policy evaluation** or prediction problem, handles the computation of the state-value function $v_{\pi}$ for an arbitrary policy $\pi$. The other problem, **policy improvement**, takes care of improving the current policy. In order to find the optimal policy, one alternates between these two processes, completing each before the other begins. This process is called **generalised policy iteration**. In general, to achieve the optimal policy one does not need to wait for a process to complete before the other starts.

### Policy Evaluation

Let me start with the policy evaluation algorithm, recall that for all states $s \in \mathcal{S}$ and a policy $\pi$ one has the Bellman equation for $v_{\pi}(s)$ (not to be confused with the Bellman optimality equation)
\begin{equation}
v_{\pi}(s) = \sum_{a} \pi(a\,|\,s) \sum_{s^{\prime}, r} p(s^{\prime}, r \,|\, s,a) \big[ r+ \gamma v_{\pi}(s^{\prime}) \big] \,.
\end{equation}
$\pi(a\,|\,s)$ above denotes the probability of selecting action $a$ in state $s$ when following policy $\pi$. The existence and uniqueness of $v_{\pi}$ are guaranteed as long as $\gamma < 1$ or eventual termination of an episode is guaranteed from all states under the policy $\pi$.

There is a number of possibilities to solve these equations. The most preferred here is the iterative method called **iterative policy evaluation**. To this end one turns the Bellman equation for $v_{\pi}(s)$ into an update rule, i.e.
\begin{equation}
v_{k+1}(s) = \sum_{a} \pi(a\,|\,s) \sum_{s^{\prime}, r} p(s^{\prime}, r \,|\, s,a) \big[ r+ \gamma v_{k}(s^{\prime}) \big] \,.
\end{equation}
The initial approximation, $v_{0}(s)$, can be chosen arbitrarily for all non-terminal states $s$. The terminal states must be assigned value 0. Notice that this algorithm uses a general idea of **bootstrapping**, i.e. updating estimates of values of states based on estimates of the values of successor states. From the Bellman equation for $v_{\pi}(s)$, one can clearly see that $v_{k} = v_{\pi}$ is a fixed point. The convergence to this fixed point is guaranteed under the same conditions that guarantee the existence of $v_{\pi}$.

### Policy Improvement

The algorithm for policy improvement hinges on a result known as **policy improvement theorem** which states that for any pair of policies $\pi$ and $\pi^{\prime}$ such that
\begin{equation}
q_{\pi}(s, \pi^{\prime}(s)) \geq v_{\pi}(s) \,, \qquad \forall s \in \mathcal{S} \,,
\end{equation}
the policy $\pi^{\prime}$ must be as good as or better than $\pi$. From this it is straightforward to infer the algorithm for policy improvement; one just needs to construct a new policy, $\pi^{\prime}$, which selects actions more greedily with respect to $v_{\pi}$ than the original policy. This new policy clearly fulfils the conditions of the policy improvement theorem and thus is better than or the same as the original policy.

As an example one can take the greedy policy, which picks the best possible action with respect to the current value function, defined by
\begin{equation}
\pi^{\prime}(s) = {\arg\max}_{a} q_{\pi}(s, a) \,, \qquad \forall s \in \mathcal{S} \,.
\end{equation}
Now, consider the case in which this greedy policy is as good as but not better than the old one. Then the value function for these two policies must be equal, i.e. $v_{\pi^{\prime}} = v_{\pi}$, which leads to the Bellman optimality equation
\begin{equation}
\begin{split}
v_{\pi^{\prime}}(s) &= \sum_{a} \pi^{\prime}(a\,|\,s) q_{\pi^{\prime}}(s, a) \\
    &= \sum_{a} \pi^{\prime}(a\,|\,s) q_{\pi}(s, a) \\
    &= \max_{a} q_{\pi}(s, a) \\
    &=\max_{a} \sum_{s^{\prime}, r} p(s^{\prime}, r \,|\, s,a) \big[ r + \gamma v_{\pi}(s^{\prime}) \big] \\
    &=\max_{a} \sum_{s^{\prime}, r} p(s^{\prime}, r \,|\, s,a) \big[ r + \gamma v_{\pi^{\prime}}(s^{\prime}) \big] \,,
\end{split}
\end{equation}
for all $s \in \mathcal{S}$. Hence, both policies must be optimal.

### Policy Iteration

Now one can combine the two above algorithms to build **policy iteration** algorithm.
- start with an arbitrary policy $\pi$
- calculate its the value function, $v_{\pi}$, with the policy evaluation algorithm (only approximately, since the precise calculation would require an infinite number of iteration steps)
- improve $\pi$ with the policy improvement algorithm to get a better policy $\pi^{\prime}$
- calculate $v_{\pi^{\prime}}$

Continue until a desirable (good enough) policy is obtained. This algorithm converges to the optimal policy $\pi^{*}$.

### Value Iteration

As was already suggested, one does not need to wait until the policy evaluation converges close enough to the value function. The process can be stopped right after the first iteration, giving an algorithm called **value iteration**. This has a particularly simple update step, namely
\begin{equation}
v_{k+1}(s) = \max_{a} \sum_{s^{\prime}, r} p(s^{\prime}, r \,|\, s,a) \big[ r + \gamma v_{k}(s^{\prime}) \big] \,.
\end{equation}

So far so good; let me now I present a short intro into the GridWorld environment used in all examples of this notebook.

## Digression: GridWorld Environment

Before starting with the examples, let me first introduce the GridWorld environment I will be using in the examples of this notebook.

I will use two classes from the file GridWorld.py, namely Worlds and GridWorld. The **class Worlds** contains several pre-defined worlds (cliff, mirror, maze, empty, track). Each of which is a numpy array of characters denoting the states. The constructor of this class does not take any arguments; and for getting a world one needs to use get member function which takes one argument, the name of the world.

There are 6 possible states and their corresponding characters:
- d - default state;
- s - start state;
- t - trap state, upon accessing the agent receives the trap reward;
- c - cliff state, upon accessing the agent receives the cliff reward and is put at one of the start states;
- g - goal state, results in termination of the episode and the agent receives the goal reward;
- w - wall state, the agent can't access this state; actions which would lead to this state do not change agent's position and return a penalty for hitting the wall.

The precise values of rewards and penalties are defined in the constructor of GridWorld. More precisely, the **GridWorld class** constructor takes four arguments:
- world - one of the worlds returned by the class Worlds;
- rewards - 5-element 1D array containing the goal, cliff, trap rewards, the step cost, and penalty for hitting the wall; the default values are [100,-50,-10,-1,-5];
- useGUI - this is a boolean variable to indicate if a GUI should be used, it can be changed later;
- name - a string to be displayed in the window's bar.

A set of wall states is automatically added around the whole world in the constructor. The member functions for interacting with the GridWorld are:
- reset - Resets the environment, i.e. move the agent to one of the start states, and sets the attribute done to False.
- step - Performs selected action which must be one of the following characters: N, S, W, E. It returns agent's new coordinates, reward, done, and info.
- render - If useGUI is True it displays the GUI; to see all agent's moves one needs to call it after every step. If action-value function is provided, the method displays agent's preferred actions, and state values in shades of green.
- set_useGUI - Sets the boolean attribute useGUI.
- get_shape - Returns the shape of the world array.
- get_agent_coos - Returns agent's coordinates.
- is_done - Returns a boolean variable indicating if the episode is over.

The last class to be used is the **StateSpace class** defined in StateSpace.py. This class will make our life easier when working with action-value functions, and when choosing actions or policies to follow. Its constructor takes 2 arguments the shape of the world and a list of possible actions, the default is ['N', 'S', 'W', 'E']. The most useful member functions are:
- reset - which sets all the state-value and action-value functions to arrays of zeros.
- eps_greedy_policy - returns an action or a list of probabilities for actions based on the action-value function.
- update_actionVF - updates action-value function, either the whole function or at a particular state.
- get_actionVF - returns the action-value function.

### Example 1: cliff-walking GridWorld - Benchmarking

The Idea behind this first example is to see how well the agent will perform in an environment if he chooses a random policy.

First, let me introduce the environment, cliff-walking GridWorld, I will be using in many examples. It consists of 12x4 accessible states for agent, with the start state and the goal state in the lower left and lower right corner, respectively. In between these two states there are 10 cliff states which upon accessing return agent on the start state and give him a large negative reward. In this setting the cliff reward is -100 points, every move costs agent -1 point, and accessing the goal states yields 13 points (so that following the optimal path gives +1). The slight modification compared to the original setting from the Sutton and Barto book is penalty for hitting a wall, -5 points, resulting in faster learning and helping to avoid useless moves.

As one can imagine, the results of the random policy will be quite bad; the reason is that the agent falls many times off the cliff, and takes a long time to find the goal state.
Running this policy for 1000 runs gives an average return of -71k with standard deviation of 73k.

I don't recommend wasting too much time on this example. It is here just for the sake of completeness, especially if one would like to benchmark other environments in the GridWorld class. But if you really want to, go ahead and play around.

In [1]:
# In case you restart the kernel and wish to continue with later examples run this cell again.
import numpy as np
from StateSpace import StateSpace
from GridWorld import GridWorld, Worlds
from HelperFunctions_TSM import printTrainingReturns, printReturns, printHighestLowest

actions = np.array(['N', 'S', 'W', 'E'])
worlds = Worlds()

In [2]:
gw = GridWorld(world=worlds.get('cliff'), rewards=[13,-100,-5,-1,-5],
               name='Random', useGUI=False)

# To get shape use gw.get_shape() instead of worlds.get('cliff').shape.
# This is because GridWorld adds walls around the world.
worldShape = gw.get_shape()
stateSpace = StateSpace(worldShape, actions=actions)

In [3]:
episodes = 2
returns = np.zeros(episodes)
gw.set_useGUI(False)                # in case you would like it with visualization, set this to True

for i in range(episodes):
    gw.reset()
    coos1 = gw.get_agent_coos()
    done = False
    cum_reward = 0
    while not done:
        action = stateSpace.eps_greedy_policy(coos=coos1, eps=1, ret_probs=False)
        coos2, reward, done, _ = gw.step(action)
        cum_reward = cum_reward + reward
    returns[i] = cum_reward

printReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over 2 episodes was -32493.0 with the standard deviation 28812.0.
The highest return was -3681.0, and the lowest return was -61305.0.


## Temporal-Difference Learning

Often, to solve the problem of RL, especially when a complete knowledge of the environment is not known, it is desirable to use experience, i.e. sample sequences of states, actions, and rewards from actual or simulated interaction with an environment, as the main source of learning. **Temporal-difference** (TD) methods use this experience as well as bootstrapping. They are very similar to what is known as Monte Carlo methods, which will be considered as a special case of TD. As outlined above, TD also uses the principle of generalised policy iteration. I will start with TD prediction; I will also follow the notation outlined in the Sutton and Barto book, namely the use of capital letters for statistical variables.

### TD(0), one-step TD

Now, I would like to describe the simplest TD method. It is a prediction method, meaning that it iteratively evaluates the value function $V$ where a policy $\pi$ is being followed (to prevent a cluttered notation I got rid of the subscript $\pi$). Essentially, the algorithm instructs the agent to take an action, $A_{t}$, given by the policy, $\pi$, which moves the agent into a new state, $S_{t+1}$, and returns the reward $R_{t+1}$.

At each time-step, this simplest TD method updates its estimates of state-value function as follows
\begin{equation}
V(S_{t}) \leftarrow V(S_{t}) + \alpha \big[ R_{t+1} + \gamma V(S_{t+1}) - V(S_{t}) \big] \,,
\end{equation}
where $\alpha$ is known as learning rate. Notice the term in the square bracket, the first two terms are the reward just returned and the discounted value of the new state. The last term is negative of value of the old state. Hence, the bracket contains the error between an updated version of $V(S_{t})$ and the old version; the whole update moves $V(S_{t})$ in the direction of minimising the error. This TD method is called **TD(0)** or **one-step TD**, because it looks only one state ahead in order to update the state-value function. The error in the square bracket is usually referred to as the **TD error** and is denoted by
\begin{equation}
\delta_{t} = R_{t+1} + \gamma V(S_{t+1}) - V(S_{t}) \,.
\end{equation}

This prediction method can be combined with the policy improvement method to obtain a full algorithm for solving RL problems.

### SARSA; on-policy, one-step TD control

All algorithms considered so far used for learning state-value function rather than action-value function. To solve the full RL problem (policy evaluation and policy improvement) with the state-value function requires the knowledge of all transition probabilities between states. All this si in principle known for GridWorld, however, it is quite cumbersome to obtain in explicit forms (and I'm also lazy :D). It is much easier to work with algorithms using the action-value function; SARSA is the first such algorithm I present.

It is basically identical to the TD(0) algorithm with the only difference that it uses transitions from one state-action pair to another rather than transitions from a state to another state. Hence, the update rule for action-value function is
\begin{equation}
Q(S_{t}, A_{t}) \leftarrow Q(S_{t}, A_{t}) + \alpha \big[ R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_{t}, A_{t}) \big] \,.
\end{equation}
Again the quantity in the square brackets is called TD error. The name for this algorithm comes from the sequence of states, actions, and the corresponding reward, i.e. $(S_{t}, A_{t}, R_{t+1}, S_{t+1}, A_{t+1})$.

This is also the first algorithm I will fully implement in the example 2 below.

### Example 2: cliff-walking with SARSA; on-policy, one-step TD control

In the previous exercise, I benchmarked the cliff-walking GridWorld environment with a random policy. Here, I will implement the SARSA algorithm. Let's see how much better it gets compared to the random policy.

I encourage the reader to play around with the parameters for rewards, learning rate, discount or greediness epsilon. Since the visualisation imposes a huge overhead, I use GUI to display the agent's moves only for each 200th episode (starting with the first one).

In [4]:
gw = GridWorld(world=worlds.get('cliff'), rewards=[13,-100,-5,-1,-5],
               name='SARSA; on-policy, one-step TD', useGUI=False)
worldShape = gw.get_shape()
stateSpace = StateSpace(worldShape, actions=actions)

In [5]:
discount = 0.9
epsilon = 0.3
episodes = 2000
returns = np.zeros(episodes)

for i in range(episodes):
    if i%200 == 0: gw.set_useGUI(True)           # render every 200-th episode, start with the first
    else: gw.set_useGUI(False)    
    learning_rate = 0.2 / 2**(i//(episodes/5))  # divide by 2 after each 20% of training
    cum_reward = 0
    gw.reset()
    done = gw.is_done()
    coos_1 = gw.get_agent_coos()
    action_1 = stateSpace.eps_greedy_policy(coos=coos_1, eps=epsilon, ret_probs=False)

    while not done:
        coos_2, reward, done, info = gw.step(action_1)
        action_2 = stateSpace.eps_greedy_policy(coos=coos_2, eps=epsilon, ret_probs=False)
        cum_reward = cum_reward + reward

        actionValue_1 = stateSpace.get_actionVF()[coos_1][action_1 == actions]
        actionValue_2 = stateSpace.get_actionVF()[coos_2][action_2 == actions]
        updatedAV_1 = actionValue_1 + learning_rate*(reward + discount*actionValue_2 - actionValue_1)
        stateSpace.update_actionVF(value=updatedAV_1, coos=coos_1, action=action_1)

        coos_1, action_1 = coos_2, action_2
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward

printTrainingReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over: 
 - the first 10% of the training was -79.5 with the standard deviation 219.0;
 - 10% to 50% of the training was -38.2 with the standard deviation 54.4;
 - 50% to 90% of the training was -30.6 with the standard deviation 42.4;
 - the last 10% of the training was -35.0 with the standard deviation 53.6.
The highest return was -1.0, and the lowest return was -2764.0.


In the next cell, the agent chooses its actions according to the greedy policy rather than the $\epsilon$-greedy policy. This will demonstrate, that the agent learned the safest possible path.

In [6]:
epsilon = 0
episodes = 100
returns = np.zeros(episodes)

for i in range(episodes):
    if i%20 == 0: gw.set_useGUI(True)           # render every 20-th episode, start with the first
    else: gw.set_useGUI(False)
    cum_reward = 0
    gw.reset()
    done = gw.is_done()

    while not done:
        coos_1 = gw.get_agent_coos()
        action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
        coos_2, reward, done, info = gw.step(action_1)
        cum_reward = cum_reward + reward
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward

printReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over 100 episodes was -3.0 with the standard deviation 0.0.
The highest return was -3.0, and the lowest return was -3.0.


### Q-learning; off-policy, one-step TD control

Let me now introduce one of the off-policy algorithms for TD control. A general difference between on-policy and off-policy methods is that an on-policy method evaluates or improves the policy being followed. Whereas, on the other hand, an off-policy method attempts to evaluate or improve a policy other than the one generating the data.

The Q-learning algorithm is defined by the following update
\begin{equation}
Q(S_{t}, A_{t}) \leftarrow Q(S_{t}, A_{t}) + \alpha \big[ R_{t+1} + \gamma \max_{a} Q(S_{t+1}, a) - Q(S_{t}, A_{t}) \big] \,.
\end{equation}
The maximum over actions hints that this algorithm will directly approximate the optimal policy $q^{*}$, independent of the policy followed. The convergence is guaranteed if all state-action pairs continue to be visited and updated. For more details I again refer the reader to the book mentioned at the beginning of this notebook.

### Example 3: cliff-walking with Q-learning; off-policy, one-step TD control

This example implements the above Q-learning algorithm; it is insightful to compare the results obtained here with those obtained in the previous example (SARSA), both in the cliff-walking world.

I again encourage the reader to play around with the parameters for rewards, learning rate, discount or greediness epsilon.

In [7]:
gw = GridWorld(world=worlds.get('cliff'), rewards=[13,-100,-5,-1,-5],
               name='Q-learning; off-policy, one-step TD', useGUI=True)
worldShape = gw.get_shape()
stateSpace = StateSpace(worldShape, actions=actions)

In [8]:
discount = 0.9
epsilon = 0.3
episodes = 1000
returns = np.zeros(episodes)

for i in range(episodes):
    if i%100 == 0: gw.set_useGUI(True)           # render every 100-th episode, start with the first
    else: gw.set_useGUI(False)
    
    learning_rate = 0.2 / 2**(i//(episodes/5))  # divide by 2 after each 20% of training
    cum_reward = 0

    gw.reset()
    done = gw.is_done()
    while not done:
        coos_1 = gw.get_agent_coos()
        action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
        coos_2, reward, done, info = gw.step(action_1)
        cum_reward = cum_reward + reward
        
        actionValue_1 = stateSpace.get_actionVF()[coos_1][action_1 == actions]
        max_AV_2 = np.amax(stateSpace.get_actionVF()[coos_2])
        updatedAV_1 = actionValue_1 + learning_rate*(reward + discount*max_AV_2 - actionValue_1)
        stateSpace.update_actionVF(updatedAV_1, coos_1, action_1)
        
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward
    
printTrainingReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over: 
 - the first 10% of the training was -237.7 with the standard deviation 419.2;
 - 10% to 50% of the training was -230.8 with the standard deviation 285.1;
 - 50% to 90% of the training was -203.1 with the standard deviation 248.0;
 - the last 10% of the training was -219.8 with the standard deviation 219.2.
The highest return was 1.0, and the lowest return was -3842.0.


The result is approximately 5-times worse than SARSA. This is because the agent learned the optimal policy which is walking along the cliff; unfortunately the followed policy is $\epsilon$-greedy which sometimes makes the agent jump off the cliff.
It is also instructive to see the agent following the greedy policy to confirm what I stated above.

In [9]:
epsilon = 0
episodes = 100
returns = np.zeros(episodes)

for i in range(episodes):
    if i%20 == 0: gw.set_useGUI(True)           # render every 20-th episode, start with the first
    else: gw.set_useGUI(False)
    cum_reward = 0
    gw.reset()
    done = gw.is_done()

    while not done:
        coos_1 = gw.get_agent_coos()
        action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
        coos_2, reward, done, info = gw.step(action_1)
        cum_reward = cum_reward + reward
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward

printReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over 100 episodes was 1.0 with the standard deviation 0.0.
The highest return was 1.0, and the lowest return was 1.0.


### Expected SARSA; one-step TD control

A slight modification to the SARSA yields an algorithm called expected SARSA. Its update rule is as follows
\begin{equation}
Q(S_{t}, A_{t}) \leftarrow Q(S_{t}, A_{t}) + \alpha \big[ R_{t+1} + \gamma \sum_{a} \pi(a \,|\, S_{t+1})Q(S_{t+1}, a) - Q(S_{t}, A_{t}) \big] \,.
\end{equation}

### Example 4: cliff-walking with Expected SARSA; one-step TD control

Although, this algorithm is more computationally complex, it seems that an agent is able to learn different policies based on the parameter $\epsilon$. I encourage the reader to experiment with different values for $\epsilon$.

In [10]:
gw = GridWorld(world=worlds.get('cliff'), rewards=[13,-100,-5,-1,-5],
               name='Expected SARSA', useGUI=True)
worldShape = gw.get_shape()
stateSpace = StateSpace(worldShape, actions=actions)

In [11]:
discount = 0.9
epsilon = 0.3
episodes = 1000
returns = np.zeros(episodes)

for i in range(episodes):
    if i%100 == 0: gw.set_useGUI(True)              # render every 100-th episode, start with the first
    else: gw.set_useGUI(False)
    
    learning_rate = 0.2 / 2**(i//(episodes/5))      # divide by 2 after each 20% of training
    cum_reward = 0
    
    gw.reset()
    done = gw.is_done()
    while not done:
        coos_1 = gw.get_agent_coos()
        action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
        coos_2, reward, done, info = gw.step(action_1)
        probs_2 = stateSpace.eps_greedy_policy(coos_2, epsilon, ret_probs=True)
        cum_reward = cum_reward + reward
        
        actionValue_1 = stateSpace.get_actionVF()[coos_1][action_1 == actions]
        expected_AV_2 = np.dot(probs_2, stateSpace.get_actionVF()[coos_2])
        updatedAV_1 = actionValue_1 + learning_rate*(reward + discount*expected_AV_2 - actionValue_1)
        stateSpace.update_actionVF(updatedAV_1, coos_1, action_1)
        
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward
    
printTrainingReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over: 
 - the first 10% of the training was -88.3 with the standard deviation 242.0;
 - 10% to 50% of the training was -34.1 with the standard deviation 49.7;
 - 50% to 90% of the training was -37.2 with the standard deviation 56.1;
 - the last 10% of the training was -27.4 with the standard deviation 36.8.
The highest return was -1.0, and the lowest return was -2343.0.


From the 'greedy' visualization in the next cell, one can see that for $\epsilon=0.3$ the agent learns an intermediate policy between the optimal and the safe. A little bit of experimentation gives different policies for different values of $\epsilon$. One can approximately write:
* $0.00 < \epsilon \leq 0.02$ gives the greedy policy,
* $0.02 < \epsilon \leq 0.32$ gives the intermediate policy,
* $0.32 < \epsilon \leq 0.05$ gives the safe policy.

In [12]:
epsilon = 0
episodes = 100
returns = np.zeros(episodes)

for i in range(episodes):
    if i%20 == 0: gw.set_useGUI(True)           # render every 20-th episode, start with the first
    else: gw.set_useGUI(False)
    cum_reward = 0
    gw.reset()
    done = gw.is_done()

    while not done:
        coos_1 = gw.get_agent_coos()
        action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
        coos_2, reward, done, info = gw.step(action_1)
        cum_reward = cum_reward + reward
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward

printReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over 100 episodes was -1.0 with the standard deviation 0.0.
The highest return was -1.0, and the lowest return was -1.0.


## Eligibility Traces a.k.a. $\lambda$-bootstrapping

So far all the algorithms used were one-step; meaning that they looked only one step ahead before performing an update to the value function. The efficiency of a learning method can be dramatically improved by using **n-step updates**. The update to a state-value function using n-step method is defined as follows
\begin{equation}
\begin{split}
V_{t+n}(S_{t}) &= V_{t+n-1}(S_{t}) + \alpha \big[ G_{t:t+n} - V_{t+n-1}(S_{t}) \big] \,, \\
G_{t:t+n} &= \sum_{i=1}^{n} \gamma^{i-1} R_{t+i} + \gamma^{n} V_{t+n-1}(S_{t+n}) \,,
\end{split}
\end{equation}
where $G_{t:t+n}$ is called **n-step return**. Now, instead of performing only one n-step update, one can update the value function using a weighted sum of many n-step returns with weights summing up to 1. A particularly useful choice of weights is $(1-\lambda)\lambda$, where $\lambda$ is called **decay factor**; and the return corresponding to this choice of weights,
\begin{equation}
G_{t}^{\lambda} = (1-\lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_{t:t+n} \,,
\end{equation}
is called **$\lambda$-return**. It can easily be shown that if an episode terminates at a terminal state $S_{t}$, and all subsequent n-step returns are equal to $G_{t}$, the above equation takes the form of
\begin{equation}
G_{t}^{\lambda} = (1-\lambda) \sum_{n=1}^{T-t-1} \lambda^{n-1} G_{t:t+n} + \lambda^{T-t-1}G_{t} \,.
\end{equation}

### SARSA($\lambda$)

Let me present an algorithms called **SARSA($\lambda$)**, which uses the machinery I recalled in the above paragraph as well as the concept of **eligibility traces**. The eligibility trace is a vector, $z$, of the same shape as the value function.

At the beginning of the episode, eligibility trace vector (ETV) is initialised to zero. Then, at each time-step, $t$, the element of ETV corresponding to the state-action pair just visited is either
- incremented by $1$, called **accumulating trace** or
- set to $1$, called **replacing trace**;

and then fades away by $\gamma\lambda$ (a product of the discount and decay factors), more precisely
\begin{equation}
\begin{split}
z_{-1} &= 0 \,,\\
z_{t} &= \gamma\lambda z_{t-1} \,,\\
z_{t}(S_{t}, A_{t}) &= z_{t-1}(S_{t}, A_{t}) + 1 \quad \text{or}\,,\\
z_{t}(S_{t}, A_{t}) &= 1 \,.
\end{split}
\end{equation}
At each time step $t$, the action-value function is updated as follows
\begin{equation}
\begin{split}
\delta_{t} &= R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_{t}, A_{t}) \,,\\
Q(\cdot, \cdot) &\leftarrow Q(\cdot, \cdot) + \alpha\delta_{t} z_{t}(\cdot, \cdot) \,,
\end{split}
\end{equation}
where, as usual, $\delta_{t}$ denotes the TD error at the time-step, $t$. If one chooses $V(\cdot)$ rather than $Q(\cdot, \cdot)$, and eliminates the actions, $A_{t}$ and $A_{t+1}$, the resulting algorithm is the one known as **TD($\lambda$)**.

To better understand this algorithm one might consider what happens at different values of the decay parameter $\lambda$.
- if $\lambda = 0$, then the eligibility trace has precisely one non-zero component at each time, namely $z_{t}(S_{t}, A_{t})$; and only the corresponding component of the action-value function is updated, namely $Q_{t}(S_{t}, A_{t})$. Hence, the whole algorithm reduces to the one-step TD method or TD(0).
- if $0 < \lambda < 1$, then more of the preceding states are updated; but due to the decay factor, the states further back in time are updated less and less.
- if $\lambda = 1$, then the resulting algorithm precisely corresponds to discounted every-visit Monte Carlo for accumulating trace, and first-visit Monte Carlo for replacing trace.

#### Digression:
Every-visit **Monte Carlo** is an algorithm where a full episode is generated upfront, and only then the value function for each state (state-action pair) is updated according to actual returns seen during the episode. I.e. for each state, $S$ appearing in the episode compute the discounted return, $G$, following it; and then update the state-value function as follows
\begin{equation}
V(S) \leftarrow V(S) + \alpha \big[ G - V(S) \big] \,.
\end{equation}
The first-visit is analogous with the only difference that the return is calculated only for the states visited for the first time in the episode, all the other visits use the return calculated when visited for the first time.

### Example 5: mirror GridWorld with SARSA($\lambda$); replacing trace

This example puts the theoretical discussion above in work. But since the cliff-walking world was solved with one step TD methods, the more sophisticated methods wouldn't add much value, and one would have hard times to judge how much better they are.

Therefore, in this and the next example I will use mirror GridWorld rather than cliff-walking because it has much larger state space. The mirror GridWorld also contains so-called trap states, these are accessible for the agent but results in a penalty; I set the penalty to be -10. All this requires a bit more sophisticated approach than just one-step algorithms.

In [13]:
gw = GridWorld(world=worlds.get('mirror'), rewards=[100,-100,-10,-1,-5],
               name='SARSA with replacing trace', useGUI=True)
worldShape = gw.get_shape()
stateSpace = StateSpace(worldShape, actions=actions)

In [14]:
discount = 0.95
decay = 0.9
episodes = 3000
returns = np.zeros(episodes)

for i in range(episodes):
    if i%300 == 9: gw.set_useGUI(True)              # render every 100-th episode, start with the 10th
    else: gw.set_useGUI(False)
    cum_reward = 0
    gw.reset()
    done = gw.is_done()
    
    learning_rate = 0.2 / 2**(i//(episodes/5))      # divide by 2 after each 20% of training
    epsilon = 0.3 * (1 - (i/episodes)**2)           # diminishes from 0.3 to 0
    replacing_trace = np.zeros( (worldShape[0], worldShape[1], len(actions)) )
    
    coos_1 = gw.get_agent_coos()
    action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
    
    while not done:
        coos_2, reward, done, info = gw.step(action_1)
        action_2 = stateSpace.eps_greedy_policy(coos_2, epsilon)
        cum_reward = cum_reward + reward
        
        replacing_trace = discount*decay*replacing_trace
        replacing_trace[coos_1][action_1 == actions] = 1
        
        actionVF = stateSpace.get_actionVF()
        td_error = reward + discount*actionVF[coos_2][action_2 == actions] - actionVF[coos_1][action_1 == actions]
        actionVF = actionVF + learning_rate*td_error*replacing_trace
        stateSpace.update_actionVF(value=actionVF)

        coos_1, action_1 = coos_2, action_2
        
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward
    
printTrainingReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over: 
 - the first 10% of the training was -63.5 with the standard deviation 726.1;
 - 10% to 50% of the training was 40.2 with the standard deviation 36.7;
 - 50% to 90% of the training was 58.1 with the standard deviation 17.3;
 - the last 10% of the training was 71.3 with the standard deviation 8.5.
The highest return was 80.0, and the lowest return was -11002.0.


If you try to find the optimal path yourself, you will see that each start state has precisely one optimal path with return of 82. Here the agent learns a policy with an average return being roughly between 65 and 75 per episode, this indicates that it is actually a decent solution. However, one can do better!

### True online SARSA($\lambda$) (dutch trace)

The **true online TD($\lambda$)** is currently the best performing TD algorithm (this holds only for tabular methods and for linear function approximation). To derive the full theoretical understanding of this algorithm is a bit messy. (Alright, you guessed it. I'm way too lazy to write it here. :D ) Hence, I will only state the update rules below and leave the experimenting and comparing with other algorithms presented in this notebook to the reader. For further explanation and references the reader is welcomed to consult Sutton and Barto book or the actual [paper](https://arxiv.org/abs/1512.04087) by Seijen et al. where this algorithm was introduced.

As usual, ETV is initialized to zero at the beginning of each episode; then at each time step, $t$, it is updated as follows
\begin{equation}
\begin{split}
z_{t} &= \gamma\lambda z_{t-1} \,,\\
z_{t}(S_{t}, A_{t}) &= 1 - \alpha\gamma\lambda z_{t-1}(S_{t}, A_{t})\,.\\
\end{split}
\end{equation}
The update rule for the action-value function reads
\begin{equation}
\begin{split}
Q_{t+1} &= Q_{t} + \alpha \delta_{t} z_{t} + \alpha \big[ Q_{t}(S_{t}, A_{t}) - Q_{t-1}(S_{t}, A_{t}) \big] z_{t} \,,\\
Q_{t+1}(S_{t}, A_{t}) &= Q_{t+1}(S_{t}, A_{t}) - \alpha \big[ Q_{t}(S_{t}, A_{t}) - Q_{t-1}(S_{t}, A_{t}) \big] \,,
\end{split}
\end{equation}
where $\delta_{t}$ is as usual.

### Example 6: mirror GridWorld with true online SARSA($\lambda$); dutch trace

And the actual implementation follows.

In [15]:
gw = GridWorld(world=worlds.get('mirror'), rewards=[100,-100,-10,-1,-5],
               name='True online SARSA', useGUI=True)
worldShape = gw.get_shape()
stateSpace = StateSpace(worldShape, actions=actions)

In [16]:
discount = 0.95
decay = 0.9
episodes = 3000
returns = np.zeros(episodes)

for i in range(episodes):
    if i%300 == 9: gw.set_useGUI(True)              # render every 100-th episode, start with the 10th
    else: gw.set_useGUI(False)
    cum_reward = 0

    gw.reset()
    done = gw.is_done()
    
    learning_rate = 0.2 / 2**(i//(episodes/5))      # divide by 2 after each 20% of training
    epsilon = 0.3 * (1 - (i/episodes)**2)           # diminishes from 0.3 to 0

    dutch_trace = np.zeros( (worldShape[0], worldShape[1], len(actions)) )
    old_actionVF = stateSpace.get_actionVF()
    
    coos_1 = gw.get_agent_coos()
    action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
    
    while not done:
        coos_2, reward, done, info = gw.step(action_1)
        action_2 = stateSpace.eps_greedy_policy(coos_2, epsilon)
        cum_reward = cum_reward + reward
        
        # dutch trace update
        temp1 = learning_rate*discount*decay*dutch_trace[coos_1][action_1 == actions]
        dutch_trace = discount*decay*dutch_trace
        dutch_trace[coos_1][action_1 == actions] = 1 - temp1
        
        # actionVF update
        actionVF = stateSpace.get_actionVF()
        td_error = reward + discount*actionVF[coos_2][action_2 == actions] \
                                    -actionVF[coos_1][action_1 == actions]
        temp2 =      actionVF[coos_1][action_1 == actions] \
                -old_actionVF[coos_1][action_1 == actions]
        actionVF = actionVF + learning_rate*(td_error + temp2)*dutch_trace
        actionVF[coos_1][action_1 == actions] = actionVF[coos_1][action_1 == actions] - learning_rate*temp2
        
        # save the current actionVF to the old one and update it
        old_actionVF = stateSpace.get_actionVF()
        stateSpace.update_actionVF(value=actionVF)
        
        coos_1, action_1 = coos_2, action_2
        
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward

printTrainingReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over: 
 - the first 10% of the training was -37.2 with the standard deviation 357.9;
 - 10% to 50% of the training was 34.1 with the standard deviation 58.5;
 - 50% to 90% of the training was 60.3 with the standard deviation 23.9;
 - the last 10% of the training was 74.0 with the standard deviation 10.4.
The highest return was 82.0, and the lowest return was -4654.0.


This algorithm gives a slightly better result for the final policy with an average return being roughly between 68-78 per episode; but more importantly it gives much better results during training. To see that, one can compare the intermediate results, namely after first 10%, between 10%-50%, and between 50%-90% of this algorithm (true online SARSA) and the plain SARSA.

### Example 7: The Maze GridWorld.

It is also insightful to play around with the other worlds, especially with the maze. I recommend to change discount from 1.0 to e.g. 0.8 and see what effect it has on delayed rewards.

In [17]:
gw = GridWorld(world=worlds.get('maze'), rewards=[100,-100,-10,-1,-5],
               name='The Maze', useGUI=True)
worldShape = gw.get_shape()
stateSpace = StateSpace(worldShape, actions=actions)

In [18]:
discount = 1
decay = 0.5
episodes = 2500

returns = np.zeros(episodes)

for i in range(episodes):
    if i%250 == 149: gw.set_useGUI(True)           # render every 250-th episode, start with the 150th
    else: gw.set_useGUI(False)
    cum_reward = 0

    gw.reset()
    done = gw.is_done()

    learning_rate = 0.2 / 2**(i//(episodes/5))      # divide by 2 after each 20% of training
    epsilon = 0.5 * (1.0 - (i/episodes)**2)         # diminishes from 0.3 to 0


    dutch_trace = np.zeros( (worldShape[0], worldShape[1], len(actions)) )
    old_actionVF = stateSpace.get_actionVF()
    
    coos_1 = gw.get_agent_coos()
    action_1 = stateSpace.eps_greedy_policy(coos_1, epsilon)
    
    while not done:
        coos_2, reward, done, info = gw.step(action_1)
        action_2 = stateSpace.eps_greedy_policy(coos_2, epsilon)
        cum_reward = cum_reward + reward
        
        # dutch trace update
        temp1 = learning_rate*discount*decay*dutch_trace[coos_1][action_1 == actions]
        dutch_trace = discount*decay*dutch_trace
        dutch_trace[coos_1][action_1 == actions] = 1 - temp1
        
        # actionVF update
        actionVF = stateSpace.get_actionVF()
        td_error = reward + discount*actionVF[coos_2][action_2 == actions] \
                                    -actionVF[coos_1][action_1 == actions]
        temp2 =      actionVF[coos_1][action_1 == actions] \
                -old_actionVF[coos_1][action_1 == actions]
        actionVF = actionVF + learning_rate*(td_error + temp2)*dutch_trace
        actionVF[coos_1][action_1 == actions] = actionVF[coos_1][action_1 == actions] - learning_rate*temp2
        
        # save the current actionVF to the old one and update it
        old_actionVF = stateSpace.get_actionVF()
        stateSpace.update_actionVF(value=actionVF)
        
        coos_1, action_1 = coos_2, action_2
        
        gw.render(actionVF=stateSpace.get_actionVF())
    returns[i] = cum_reward
    
printTrainingReturns(returns)
printHighestLowest(returns)

Performance measure: the return per episode averaged over: 
 - the first 10% of the training was -995.7 with the standard deviation 1688.6;
 - 10% to 50% of the training was -245.5 with the standard deviation 371.9;
 - 50% to 90% of the training was -40.2 with the standard deviation 123.6;
 - the last 10% of the training was 26.8 with the standard deviation 58.4.
The highest return was 80.0, and the lowest return was -19474.0.


***The end of the notebook.***